<a href="https://colab.research.google.com/github/ahatasham5/Doctor-AI/blob/main/Doctor-AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning Using Google Gemma's Model

In [1]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.8 MB/s eta 0:00:00
     ━━━

In [2]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

In [3]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

### Prerequisites
* nf4(4-bit NormalFloat(NF4)) : https://www.kaggle.com/code/lorentzyeung/what-s-4-bit-quantization-how-does-it-help-llama2


In [4]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [11]:
text = "What is medicine?,"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

What is medicine?, what is the purpose of medicine?

The purpose of medicine is to treat disease and to prevent disease.

The purpose of medicine is to treat disease and to prevent disease.

The purpose of medicine is to treat disease and to prevent disease.




In [12]:
text = "what is fever?"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

what is fever?

A 100-turn, 8.0-cm-diameter coil is oriented in a magnetic field of 0.68 T. (a) If the torque on the coil is $0.30 \mathrm{


In [13]:
os.environ["WANDB_DISABLED"] = "false"

In [14]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

In [25]:
from datasets import load_dataset
data = load_dataset("shibing624/medical", "finetune")
data = data.map(lambda samples: tokenizer(samples["instruction"]), batched=True)

Generating train split:   0%|          | 0/2066589 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2066589 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

**if you need to handle the different splits separately**

train_data = data["train"]
val_data = data["validation"]
test_data = data["test"]

data = load_dataset("shibing624/medical", "finetune")
train_data = data["train"].map(lambda samples: tokenizer(samples["instruction"]), batched=True)
val_data = data["validation"].map(lambda samples: tokenizer(samples["instruction"]), batched=True)
test_data = data["test"].map(lambda samples: tokenizer(samples["instruction"]), batched=True)

In [30]:
data['train']['instruction']

['血热的临床表现是什么?',
 '帕金森叠加综合征的辅助治疗有些什么？',
 '卵巢癌肉瘤的影像学检查有些什么？',
 '婴儿耳朵湿疹流水怎么治疗',
 '低T3综合征的并发症是什么？',
 '重复胚胎停止发育的高危因素有些什么？',
 '宫颈糜烂用什么手术治疗比较好呢？，详细病情及咨询目的:我想问一下，宫颈糜烂用什么手术治疗比较好呢？\n用凝固刀手术多少钱？',
 '弯曲菌感染的就诊科室是什么？',
 '胰岛素强化治疗的推荐药有些什么？',
 '发育性颈椎管狭窄症的辅助治疗有些什么？',
 '唇畸形的辅助治疗有些什么？',
 '房性心动过速的分类是什么?',
 '严重皮肤软组织损伤的手术治疗有些什么？',
 '青少年抑郁的高危因素有些什么？',
 '得了脂肪肝会医治不？，我叔叔最近去医院检查身体。院的医生说是得了这种病。不知晓怎么会得了这种病。不知晓哪里的医院救治这种病的疗效好。谁知道的话说说。好给叔叔说说。去那里的医院好好救治救治。也不着急。不着再忧心他。不知晓这种病好了会不会在病发了。治全愈不能够。',
 '肿瘤腹膜转移的化疗有些什么？',
 '急性前壁心肌梗塞的临床表现有些什么？',
 '大血管转位的影像学检查有些什么？',
 '蓝鼓膜与胆固醇肉芽肿的并发症是什么？',
 '关于妇科炎症是什么呢？，平时其实都还好，但是我每次月经之后就会犯病，具体症状就是私处瘙痒，小腹下坠疼痛，这个情况是有炎症吗？请问：关于妇科炎症是什么呢？',
 '老年痴呆尿毒症的临床表现有些什么？',
 'STEMI小血管病变的推荐药有些什么？',
 '抗磷脂抗体综合征怀孕是什么意思？，本人53岁，最近随着年龄的增加，最近总是腿疼，之前去过免疫科看过，但是效果不大，中西药都吃过了，尤其是天气变化就会不舒服，请问抗磷脂抗体综合征怀孕是什么意思？',
 '植物神经紊乱可以运动吗，我有植物神经紊乱，抑郁症，焦虑症，肝肾可能也不太好，因为老是腰酸背痛，夜尿多，额头总是出大粒痘痘，头疼头晕的特别特别困，最近坚持早睡早起，天天运动，但是还是特别困，运动完后觉得自己特别特别累，又困，困的都抬不起眼皮了，我怀疑是脑供血不足。该怎么办呢，到底能不能运动啊？',
 '肾虚中医有什么食疗密方，我今年岁数也不大啊，但是总是感觉怕冷，我浑身也没有力气干不了事情，这个毛病很久了，我害怕我的肾有毛病，请问肾虚中医有

In [31]:
def formatting_func(example):
    text = f"Instruction: {example['instruction'][0]}\nOutput: {example['output'][0]}"
    return [text]

In [32]:
data['train']

Dataset({
    features: ['instruction', 'input', 'output', 'input_ids', 'attention_mask'],
    num_rows: 2066589
})

In [33]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2066589 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [34]:
trainer.train()

Step,Training Loss
1,5.204400
2,3.515500
3,4.447400
4,4.293400
5,5.796700
6,4.235900
7,4.006000
8,4.338600
9,4.411400
10,4.435200


TrainOutput(global_step=100, training_loss=3.4394597816467285, metrics={'train_runtime': 160.2227, 'train_samples_per_second': 2.497, 'train_steps_per_second': 0.624, 'total_flos': 681468765757440.0, 'train_loss': 3.4394597816467285, 'epoch': 0.19})

In [36]:
text = "Instruction: I have fever, what should i do?"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Instruction: I have fever, what should i do?
Output: Fever is a common symptom of many diseases. It is a sign of inflammation, infection, or fever. Fever is a sign of inflammation. It is a sign of infection. It is a sign of fever. It is a sign of
